#Importing the required library

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install replicate
!pip install pypdf
!pip install chromadb
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.9/357.9 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.4/581.4 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

Importing the required libraries and setting the replicate API

In [ ]:
import os
import sys
from langchain.llms import Replicate
from langchain.vectorstores import Pinecone
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import Chroma

# Replicate API token
os.environ['REPLICATE_API_TOKEN'] = "Your replicate api"


Creating a function to read the pdf and extract the text from it.

In [ ]:
from pypdf import PdfReader
def get_pdf_text(pdf):
    text = ""
    pdf_reader = PdfReader(pdf)
    for page in pdf_reader.pages:
      text += page.extract_text()
    return text


Storing the textual content of the input pdf in t

In [ ]:
t = get_pdf_text('./heart.pdf')

In [ ]:
print(t)

 
FACTSHEET: Cardiovascular diseases in India  
 
Globally…  
 Cardiovascular diseases (CVDs), commonly referred to as heart disease or stroke, are 
the number 1 cause of death around the world  
 1 in 3 deaths globally are as result of CVD, yet the majority of premature heart disease 
and stroke is preventable1 
 In 2010 CVD cost US$ 863 billion – this is estimated to rise by 22% to US$ 1,044 billion 
by 20302 
 80% of CVD deaths occur in low - to middle -income countries.  
 
In India…  
 Non-communicable diseases (NCDs), including CVDs, are estimated to account for 60% 
of total adult deaths in India  
 CVDs account for over a quarter (26 %) of these deaths  
 Some of the CVD related risks factors in adults in India are outlined below:  
 15% of the population smoke tobacco  
 4.3 litres of pure alcohol consumed per person  
 Just over a fifth (21.1 %) have hypertension  which can increase risk of heart 
attack, heart failure, kidney disease  or stroke  
Taking action…  


This code snippet splits a large text input into smaller, manageable chunks using a CharacterTextSplitter class. The splitting criteria are based on a separator (e.g., newline character) and chunk size/overlap settings. Then, it  reassembles the chunks back into documents or generates new documents based on the splitting strategy

In [ ]:
#loader = PyPDFLoader('./heart.pdf')
#documents = loader.load()
documents = get_pdf_text('./heart.pdf')
# Split the documents into smaller chunks for processing
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
chunks = text_splitter.split_text(documents)
texts= text_splitter.create_documents(chunks)
#texts = text_splitter.split_documents(documents)

Using HuggingFace Embeddings on text to and then store it in chroma vectorstore

In [ ]:
#embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
embeddings = HuggingFaceEmbeddings()
vectorstore = Chroma.from_documents(texts, embeddings,ids=None, collection_name="heart",persist_directory='./chroma_db')
vectorstore.persist()

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


Importing LLM Llama 3 from replicate tuning some of its parameters like Temperature which is used for randomness of the response by llm.
Then the system prompt that helps the llm understand its role pretty well.
Then the max tokens for restricting the size, there are more parameters to tune but these are basic.

In [ ]:
llm = Replicate(
    model="meta/meta-llama-3-8b",
    input={"temperature":0.6,
    "prompt" : "You are an helpful AI.You do not generate further questions and helpful answer of them",
    "max_tokens":2000,
    }
)

#ConversationalRetrievalChain
 It is a kind of chain used to be provided with a query and to answer it using documents retrieved from the query. It is one of the many possibilities to perform Retrieval-Augmented Generation.

But it won’t only answer your last query, it will also use the chat history to improve the quality of the RAG by taking into account past querie

In [ ]:
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(search_kwargs={'k': 2}), # Making the vector store retrievable so that the data of pdf can be used to answer the prompts . k means the retrieval system will find and return the two most relevant documents (based on some similarity metric) to the question posed by the LLM.
    return_source_documents=True # returns the source doucment also
)

In [ ]:
chat_history = []  #creating the chat history
while True:
    query = input('Prompt: ')  # taking the prompt as input
    if query.lower() in ["exit", "quit", "q"]:  # exit statements
        print('Exiting')
        sys.exit()
    result = qa_chain({'question': query, 'chat_history': chat_history})  # retrival chain to answer our prompt
    print('Answer: ' + result['answer'] + '\n')
    chat_history.append((query, result['answer']))  # appending the chat history